In [ ]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash_extensions.enrich import DashProxy, Output, Input, State
import plotly.express as px
import base64
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import asyncio

# Load environment variables from a .env file in current working directory
load_dotenv()

# Import the AnimalShelter class from the CRUD Python module
from enhanced import AnimalShelter

# Retrieve username and password from environment variables
username = os.getenv('AACUSER')
password = os.getenv('AACPASS')

if not username or not password:
    raise ValueError("Username or password not found in environment variables")

# Connect to the database via the CRUD Module
try:
    db = AnimalShelter(username, password)
except Exception as e:
    print(f"Error connecting to database: {e}")
    raise

async def fetch_data(query):
    try:
        data = await db.read(query)
        return pd.DataFrame.from_records(data)
    except Exception as e:
        print(f"Error reading from database: {e}")
        return pd.DataFrame()

async def fetch_aggregated_data(pipeline):
    try:
        data = await db.aggregate(pipeline)
        return pd.DataFrame.from_records(data)
    except Exception as e:
        print(f"Error aggregating from database: {e}")
        return pd.DataFrame()

# Initialize the JupyterDash app
app = DashProxy(__name__)

# Encode the logo image for inclusion in the app layout
image_filename = 'graziosoSalvareLogo.png'
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
except FileNotFoundError:
    print(f"Error: Image file {image_filename} not found.")
    encoded_image = None

# Define the layout of the app
app.layout = html.Div([
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image.decode()}')) if encoded_image else html.Center(html.H1('Logo not found')),
    html.Center(html.B(html.H1('Mark McCormack CS 340 Project Two Dashboard'))),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'W'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MW'},
                {'label': 'Disaster & Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset to default', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
        ),
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[],
        data=[],
        page_current=0,
        page_size=10,
        page_action="native",
        filter_action="native",
        row_selectable="single",
        sort_action='native',
        sort_mode='single',
        selected_rows=[0]
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Define the callback to update the DataTable based on the filter type
@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    Input('filter-type', 'value')
)
async def update_dashboard(filter_type):
    try:
        if filter_type == 'W':
            pipeline = [
                {'$match': {'sex_upon_outcome': 'Intact Female'}},
                {'$match': {'$or': [
                    {'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesapeake Bay Retriever'},
                    {'breed': 'Newfoundland'}
                ]}},
                {'$match': {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}}
            ]
        elif filter_type == 'MW':
            pipeline = [
                {'$match': {'sex_upon_outcome': 'Intact Male'}},
                {'$match': {'$or': [
                    {'breed': 'German Shepherd'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'}
                ]}},
                {'$match': {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}}
            ]
        elif filter_type == 'DIT':
            pipeline = [
                {'$match': {'sex_upon_outcome': 'Intact Male'}},
                {'$match': {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}
                ]}},
                {'$match': {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 20}},
                    {'age_upon_outcome_in_weeks': {'$lte': 300}}
                ]}}
            ]
        else:
            pipeline = []

        df = await fetch_aggregated_data(pipeline)
        columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data = df.to_dict('records')
    except Exception as e:
        print(f"Error updating dashboard: {e}")
        columns = []
        data = []

    return data, columns

# Define the callback to update the graph based on the DataTable data
@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
async def update_graphs(viewData):
    if viewData is None:
        return [html.P("No data available")]

    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()

    return [dcc.Graph(figure=px.pie(
        data_frame=dff,
        values=values,
        names=names,
        color_discrete_sequence=px.colors.sequential.RdBu,
        width=800,
        height=500,
        title='Preferred Breeds'
    ))]

# Define the callback to update DataTable styles based on selected columns
@app.callback(Output('datatable-id', 'style_data_conditional'), [Input('datatable-id', 'selected_columns')])
async def update_styles(selected_columns):
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

# Define the callback to update the map based on the selected row in the DataTable
@app.callback(Output('map-id', "children"), [Input('datatable-id', "derived_virtual_data"), Input('datatable-id', "derived_virtual_selected_rows")])
async def update_map(viewData, index):
    if viewData is None or index is None or not index:
        return [html.P("No data available")]

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    try:
        position = [dff.iloc[row, 13], dff.iloc[row, 14]]
        breed = dff.iloc[row, 4]
        name = dff.iloc[row, 9]
    except (IndexError, KeyError) as e:
        print(f"Error fetching data for map: {e}")
        position = [30.75, -97.48]  # Default position
        breed = "Unknown"
        name = "Unknown"

    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=position, children=[
            dl.Tooltip(breed),
            dl.Popup([html.H1("Animal Name"), html.P(name)])
        ])
    ])]

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
